In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor, FeaturesData
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
import os
import shap
shap.initjs()

pd.set_option('max_rows', 200)
pd.set_option('display.max_columns', 300)
np.random.seed(666)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.5f}'.format)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

### Define functions and globals

In [2]:
TARGET_COL = 'diabetes_mellitus'
cat_cols = ['ethnicity','hospital_admit_source','icu_admit_source','icu_stay_type','icu_type',
            'age_group','demo_profile','apache_3j_group','a1c_bp_group',
            'apache_2_diagnosis','icu_id']
k = 5
seed = 8

def target_encode(var, dv, weight = 36):
    mean = dv.mean()
    true_weight = weight * max(1, mean/(1-mean))
    agg = dv.groupby(var,dropna=False).agg(['count','mean'])
    counts = agg['count']
    means = agg['mean']
    return (counts * means + true_weight) / (counts + true_weight/mean)

def target_encode_train_test(var_train, dv_train, var_test, weight = 36):
    encode_key = target_encode(var_train,dv_train,weight)
    return var_train.map(encode_key), var_test.map(encode_key).astype(np.float32).fillna(dv_train.mean())

def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':

                if (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df

### Read in data

In [3]:
train = pd.read_csv('Data/train_FE2_filter5.csv')
test = pd.read_csv('Data/test_FE2_filter5.csv')

In [ ]:
# for col in ['icu_id']:
#     train_only = list(set(train[col].unique()) - set(test[col].unique()))
#     test_only = list(set(test[col].unique()) - set(train[col].unique()))
#     both = list(set(test[col].unique()).union(set(train[col].unique())))
#     train.loc[train[col].isin(train_only), col] = np.nan
#     test.loc[test[col].isin(test_only), col] = np.nan

In [ ]:
# train_only = list(set(train['icu_id'].unique()) - set(test['icu_id'].unique()))
# test_only = list(set(test['icu_id'].unique()) - set(train['icu_id'].unique()))
# both = list(set(test['icu_id'].unique()).union(set(train['icu_id'].unique())))
# train.loc[train['icu_id'].isin(train_only), 'icu_id'] = np.nan
# test.loc[test['icu_id'].isin(test_only), 'icu_id'] = np.nan

In [4]:
train['dataset_label'] = 0
test['dataset_label'] = 1
test[TARGET_COL] = np.nan
combined = pd.concat([train, test], ignore_index=True)
str_cols = ['icu_id','apache_2_diagnosis']
combined[str_cols] = combined[str_cols].astype(str)
combined[cat_cols] = combined[cat_cols].fillna('')
# combined[cat_cols] = combined[cat_cols].astype('category')
train = combined[combined.dataset_label == 0].drop('dataset_label',axis=1)
test = combined[combined.dataset_label == 1].drop(['dataset_label',TARGET_COL],axis=1)

In [5]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 254.99 MB (48.0 % reduction)
Mem. usage decreased to 20.02 MB (48.0 % reduction)


In [6]:
features_to_ignore = ['encounter_id','hospital_id']
# these variables are target-encoded within cv-folds to prevent data leakage and overoptimistic cv performance
vars_to_encode = ['icu_id','apache_3j_diagnosis','demo_profile']
trainX = train.drop(features_to_ignore+[TARGET_COL],axis=1)
trainY = train[TARGET_COL]

### Train model

In [9]:
model_name = 'cb_.01LR_FE2_filtered5_GPU_NewtonL2_8x6_cv'
# kf = KFold(n_splits=k, random_state=seed, shuffle=True)
kf = RepeatedKFold(n_splits=8, n_repeats=6, random_state=seed)
params = {
    'objective':'Logloss'
    ,'cat_features': cat_cols
    ,'verbose': 100
    ,'iterations': 11000
    ,'learning_rate': 0.01
    ,'eval_metric': 'AUC'
    ,'one_hot_max_size': 20
    ,'depth': 8
    ,'score_function': 'NewtonL2'
    ,'l2_leaf_reg': 9
#    ,'boosting_type': 'Ordered'
    ,'task_type': 'GPU'
    ,'model_size_reg': 0
    ,'bagging_temperature': .4
}

In [10]:
best_iters = np.zeros(48)
for i, (train_index, valid_index) in enumerate(kf.split(trainX,trainY)):
    print('Fold {0}'.format(i))
    test_copy = test.copy()
    trainX_cv, trainY_cv = trainX.iloc[train_index,:], trainY[train_index]
    validX_cv, validY_cv = trainX.iloc[valid_index,:], trainY[valid_index]
#     for col in ['icu_id']:
#         trainX_cv[col] = trainX_cv[col].astype(str)
#         test_copy[col] = test_copy[col].astype(str)
#         train_only = list(set(trainX_cv[col].unique()) - set(test_copy[col].unique()))
#         test_only = list(set(test_copy[col].unique()) - set(trainX_cv[col].unique()))
#         trainX_cv.loc[trainX_cv[col].isin(train_only), col] = ''
#         test_copy.loc[test_copy[col].isin(test_only), col] = ''
    for col in vars_to_encode:
        trainX_cv[col+'_encoded'], validX_cv[col+'_encoded'] = target_encode_train_test(
            trainX_cv[col], trainY_cv, validX_cv[col])
        _, test_copy[col+'_encoded'] = target_encode_train_test(
            trainX_cv[col], trainY_cv, test_copy[col])

    trainX_cv.drop(['apache_3j_diagnosis'],axis=1,inplace=True)
    validX_cv.drop(['apache_3j_diagnosis'],axis=1,inplace=True)
    test_copy.drop(features_to_ignore+['apache_3j_diagnosis'],axis=1,inplace=True)
    assert trainX_cv.columns.equals(test_copy.columns)
    model_cv = CatBoostClassifier(**params)
    model_cv_fitted = model_cv.fit(trainX_cv, trainY_cv, eval_set = (validX_cv,validY_cv), use_best_model=False)
    best_iters[i] = model_cv.best_iteration_
    if not os.path.exists('Models/{0}'.format(model_name)):
        os.makedirs('Models/{0}'.format(model_name))
    model_cv_fitted.save_model('Models/{0}/{1}'.format(model_name,i))
pd.DataFrame(best_iters).to_csv('Models/{0}/best_iters.csv'.format(model_name),index=False)

Fold 0
0:	learn: 0.8260265	test: 0.8157097	best: 0.8157097 (0)	total: 72.6ms	remaining: 13m 18s
100:	learn: 0.8554083	test: 0.8437323	best: 0.8437323 (100)	total: 7.95s	remaining: 14m 18s
200:	learn: 0.8661002	test: 0.8533310	best: 0.8533310 (200)	total: 16.4s	remaining: 14m 40s
300:	learn: 0.8730752	test: 0.8587250	best: 0.8587250 (300)	total: 24.5s	remaining: 14m 29s
400:	learn: 0.8780731	test: 0.8617747	best: 0.8617747 (400)	total: 32.4s	remaining: 14m 16s
500:	learn: 0.8820795	test: 0.8637388	best: 0.8637388 (500)	total: 40.5s	remaining: 14m 7s
600:	learn: 0.8855414	test: 0.8652190	best: 0.8652190 (600)	total: 48.4s	remaining: 13m 56s
700:	learn: 0.8885872	test: 0.8662978	best: 0.8662978 (700)	total: 56.1s	remaining: 13m 44s
800:	learn: 0.8914303	test: 0.8672329	best: 0.8672329 (800)	total: 1m 4s	remaining: 13m 35s
900:	learn: 0.8940161	test: 0.8681384	best: 0.8681384 (900)	total: 1m 11s	remaining: 13m 26s
1000:	learn: 0.8964166	test: 0.8688716	best: 0.8688716 (1000)	total: 1m 19s	

8800:	learn: 0.9830773	test: 0.8746109	best: 0.8746941 (8148)	total: 12m 1s	remaining: 3m
8900:	learn: 0.9835284	test: 0.8745925	best: 0.8746941 (8148)	total: 12m 10s	remaining: 2m 52s
9000:	learn: 0.9839819	test: 0.8745478	best: 0.8746941 (8148)	total: 12m 23s	remaining: 2m 45s
9100:	learn: 0.9844154	test: 0.8745093	best: 0.8746941 (8148)	total: 12m 31s	remaining: 2m 36s
9200:	learn: 0.9848375	test: 0.8744967	best: 0.8746941 (8148)	total: 12m 40s	remaining: 2m 28s
9300:	learn: 0.9852390	test: 0.8744912	best: 0.8746941 (8148)	total: 12m 48s	remaining: 2m 20s
9400:	learn: 0.9856651	test: 0.8744240	best: 0.8746941 (8148)	total: 12m 56s	remaining: 2m 12s
9500:	learn: 0.9860632	test: 0.8744159	best: 0.8746941 (8148)	total: 13m 4s	remaining: 2m 3s
9600:	learn: 0.9864565	test: 0.8743773	best: 0.8746941 (8148)	total: 13m 12s	remaining: 1m 55s
9700:	learn: 0.9868421	test: 0.8743431	best: 0.8746941 (8148)	total: 13m 20s	remaining: 1m 47s
9800:	learn: 0.9872250	test: 0.8742653	best: 0.8746941 (8

6400:	learn: 0.9673172	test: 0.8852386	best: 0.8852410 (6382)	total: 9m 10s	remaining: 6m 35s
6500:	learn: 0.9680890	test: 0.8852516	best: 0.8852597 (6492)	total: 9m 20s	remaining: 6m 27s
6600:	learn: 0.9688283	test: 0.8852746	best: 0.8852888 (6594)	total: 9m 31s	remaining: 6m 20s
6700:	learn: 0.9695567	test: 0.8853033	best: 0.8853074 (6683)	total: 9m 41s	remaining: 6m 13s
6800:	learn: 0.9703138	test: 0.8852932	best: 0.8853074 (6683)	total: 9m 52s	remaining: 6m 5s
6900:	learn: 0.9709755	test: 0.8853036	best: 0.8853109 (6851)	total: 10m 2s	remaining: 5m 57s
7000:	learn: 0.9716946	test: 0.8852637	best: 0.8853162 (6932)	total: 10m 13s	remaining: 5m 50s
7100:	learn: 0.9723656	test: 0.8852816	best: 0.8853162 (6932)	total: 10m 24s	remaining: 5m 42s
7200:	learn: 0.9730745	test: 0.8853049	best: 0.8853162 (6932)	total: 10m 35s	remaining: 5m 35s
7300:	learn: 0.9737661	test: 0.8853164	best: 0.8853454 (7267)	total: 10m 46s	remaining: 5m 27s
7400:	learn: 0.9743909	test: 0.8853287	best: 0.8853454 (7

4000:	learn: 0.9442199	test: 0.8811145	best: 0.8811145 (4000)	total: 5m 29s	remaining: 9m 36s
4100:	learn: 0.9453708	test: 0.8811647	best: 0.8811663 (4096)	total: 5m 37s	remaining: 9m 27s
4200:	learn: 0.9465888	test: 0.8811954	best: 0.8811959 (4198)	total: 5m 45s	remaining: 9m 19s
4300:	learn: 0.9477002	test: 0.8812032	best: 0.8812215 (4255)	total: 5m 53s	remaining: 9m 11s
4400:	learn: 0.9488246	test: 0.8812609	best: 0.8812636 (4373)	total: 6m 2s	remaining: 9m 2s
4500:	learn: 0.9499697	test: 0.8812865	best: 0.8812928 (4497)	total: 6m 10s	remaining: 8m 54s
4600:	learn: 0.9510390	test: 0.8812857	best: 0.8812928 (4497)	total: 6m 18s	remaining: 8m 46s
4700:	learn: 0.9521150	test: 0.8813125	best: 0.8813136 (4693)	total: 6m 26s	remaining: 8m 38s
4800:	learn: 0.9531597	test: 0.8813308	best: 0.8813368 (4798)	total: 6m 35s	remaining: 8m 30s
4900:	learn: 0.9542027	test: 0.8813757	best: 0.8813838 (4896)	total: 6m 43s	remaining: 8m 21s
5000:	learn: 0.9552006	test: 0.8814109	best: 0.8814233 (4988)	

1600:	learn: 0.9082072	test: 0.8778997	best: 0.8778997 (1600)	total: 2m 48s	remaining: 16m 27s
1700:	learn: 0.9100360	test: 0.8781780	best: 0.8781780 (1700)	total: 2m 59s	remaining: 16m 19s
1800:	learn: 0.9118604	test: 0.8784682	best: 0.8784682 (1800)	total: 3m 10s	remaining: 16m 10s
1900:	learn: 0.9136358	test: 0.8787189	best: 0.8787189 (1900)	total: 3m 20s	remaining: 16m 1s
2000:	learn: 0.9153835	test: 0.8789044	best: 0.8789058 (1996)	total: 3m 31s	remaining: 15m 50s
2100:	learn: 0.9170861	test: 0.8790985	best: 0.8790985 (2100)	total: 3m 43s	remaining: 15m 44s
2200:	learn: 0.9187717	test: 0.8792595	best: 0.8792611 (2198)	total: 3m 54s	remaining: 15m 38s
2300:	learn: 0.9204323	test: 0.8794156	best: 0.8794156 (2299)	total: 4m 2s	remaining: 15m 17s
2400:	learn: 0.9220703	test: 0.8796026	best: 0.8796026 (2400)	total: 4m 10s	remaining: 14m 57s
2500:	learn: 0.9236186	test: 0.8797285	best: 0.8797379 (2488)	total: 4m 18s	remaining: 14m 38s
2600:	learn: 0.9251705	test: 0.8798886	best: 0.87989

...

2800:	learn: 0.9283109	test: 0.8797395	best: 0.8797395 (2800)	total: 2m 44s	remaining: 8m
2900:	learn: 0.9298092	test: 0.8798919	best: 0.8798919 (2900)	total: 2m 50s	remaining: 7m 54s
3000:	learn: 0.9312567	test: 0.8799658	best: 0.8799662 (2997)	total: 2m 55s	remaining: 7m 48s
3100:	learn: 0.9326898	test: 0.8800217	best: 0.8800276 (3094)	total: 3m 1s	remaining: 7m 42s
3200:	learn: 0.9340956	test: 0.8801083	best: 0.8801083 (3200)	total: 3m 7s	remaining: 7m 36s
3300:	learn: 0.9354865	test: 0.8802044	best: 0.8802044 (3300)	total: 3m 13s	remaining: 7m 30s
3400:	learn: 0.9368492	test: 0.8803484	best: 0.8803523 (3395)	total: 3m 19s	remaining: 7m 24s
3500:	learn: 0.9381578	test: 0.8804247	best: 0.8804259 (3481)	total: 3m 24s	remaining: 7m 18s
3600:	learn: 0.9394085	test: 0.8805104	best: 0.8805169 (3572)	total: 3m 30s	remaining: 7m 12s
3700:	learn: 0.9407237	test: 0.8805411	best: 0.8805500 (3682)	total: 3m 36s	remaining: 7m 6s
3800:	learn: 0.9419911	test: 0.8806613	best: 0.8806675 (3797)	total

500:	learn: 0.8817824	test: 0.8687733	best: 0.8687733 (500)	total: 29.9s	remaining: 10m 26s
600:	learn: 0.8852031	test: 0.8703297	best: 0.8703297 (600)	total: 35.7s	remaining: 10m 16s
700:	learn: 0.8882315	test: 0.8715120	best: 0.8715120 (700)	total: 41.5s	remaining: 10m 9s
800:	learn: 0.8910494	test: 0.8724886	best: 0.8724886 (800)	total: 47.4s	remaining: 10m 3s
900:	learn: 0.8936293	test: 0.8733801	best: 0.8733801 (900)	total: 53.3s	remaining: 9m 57s
1000:	learn: 0.8960156	test: 0.8740484	best: 0.8740484 (1000)	total: 59.1s	remaining: 9m 50s
1100:	learn: 0.8982782	test: 0.8745646	best: 0.8745646 (1100)	total: 1m 5s	remaining: 9m 53s
1200:	learn: 0.9003192	test: 0.8750104	best: 0.8750104 (1200)	total: 1m 13s	remaining: 10m 2s
1300:	learn: 0.9023181	test: 0.8753972	best: 0.8753979 (1298)	total: 1m 21s	remaining: 10m 10s
1400:	learn: 0.9042665	test: 0.8758932	best: 0.8758932 (1400)	total: 1m 29s	remaining: 10m 16s
1500:	learn: 0.9061235	test: 0.8762429	best: 0.8762439 (1499)	total: 1m 3

9300:	learn: 0.9847848	test: 0.8806893	best: 0.8807449 (8269)	total: 12m 44s	remaining: 2m 19s
9400:	learn: 0.9852412	test: 0.8807076	best: 0.8807449 (8269)	total: 12m 50s	remaining: 2m 11s
9500:	learn: 0.9856460	test: 0.8806725	best: 0.8807449 (8269)	total: 12m 56s	remaining: 2m 2s
9600:	learn: 0.9860192	test: 0.8806611	best: 0.8807449 (8269)	total: 13m 2s	remaining: 1m 54s
9700:	learn: 0.9864246	test: 0.8806856	best: 0.8807449 (8269)	total: 13m 8s	remaining: 1m 45s
9800:	learn: 0.9868203	test: 0.8806236	best: 0.8807449 (8269)	total: 13m 14s	remaining: 1m 37s
9900:	learn: 0.9871970	test: 0.8806288	best: 0.8807449 (8269)	total: 13m 20s	remaining: 1m 28s
10000:	learn: 0.9875656	test: 0.8806333	best: 0.8807449 (8269)	total: 13m 26s	remaining: 1m 20s
10100:	learn: 0.9879317	test: 0.8806312	best: 0.8807449 (8269)	total: 13m 33s	remaining: 1m 12s
10200:	learn: 0.9882893	test: 0.8806314	best: 0.8807449 (8269)	total: 13m 39s	remaining: 1m 4s
10300:	learn: 0.9886310	test: 0.8806106	best: 0.880

7000:	learn: 0.9722964	test: 0.8722087	best: 0.8722410 (6718)	total: 6m 52s	remaining: 3m 55s
7100:	learn: 0.9729871	test: 0.8721957	best: 0.8722410 (6718)	total: 6m 57s	remaining: 3m 49s
7200:	learn: 0.9736304	test: 0.8721983	best: 0.8722410 (6718)	total: 7m 4s	remaining: 3m 43s
7300:	learn: 0.9743029	test: 0.8721696	best: 0.8722410 (6718)	total: 7m 10s	remaining: 3m 37s
7400:	learn: 0.9749364	test: 0.8721747	best: 0.8722410 (6718)	total: 7m 16s	remaining: 3m 32s
7500:	learn: 0.9755429	test: 0.8721633	best: 0.8722410 (6718)	total: 7m 21s	remaining: 3m 26s
7600:	learn: 0.9761499	test: 0.8721386	best: 0.8722410 (6718)	total: 7m 27s	remaining: 3m 20s
7700:	learn: 0.9766960	test: 0.8721015	best: 0.8722410 (6718)	total: 7m 33s	remaining: 3m 14s
7800:	learn: 0.9772679	test: 0.8720912	best: 0.8722410 (6718)	total: 7m 39s	remaining: 3m 8s
7900:	learn: 0.9778320	test: 0.8721047	best: 0.8722410 (6718)	total: 7m 45s	remaining: 3m 2s
8000:	learn: 0.9783785	test: 0.8720991	best: 0.8722410 (6718)	t

4700:	learn: 0.9521204	test: 0.8877753	best: 0.8877985 (4594)	total: 4m 35s	remaining: 6m 8s
4800:	learn: 0.9531971	test: 0.8877895	best: 0.8877985 (4594)	total: 4m 41s	remaining: 6m 2s
4900:	learn: 0.9542547	test: 0.8877317	best: 0.8878121 (4824)	total: 4m 47s	remaining: 5m 57s
5000:	learn: 0.9552966	test: 0.8877155	best: 0.8878121 (4824)	total: 4m 52s	remaining: 5m 51s
5100:	learn: 0.9562884	test: 0.8876957	best: 0.8878121 (4824)	total: 4m 58s	remaining: 5m 45s
5200:	learn: 0.9572836	test: 0.8877513	best: 0.8878121 (4824)	total: 5m 4s	remaining: 5m 39s
5300:	learn: 0.9582690	test: 0.8877046	best: 0.8878121 (4824)	total: 5m 10s	remaining: 5m 33s
5400:	learn: 0.9592476	test: 0.8876985	best: 0.8878121 (4824)	total: 5m 16s	remaining: 5m 27s
5500:	learn: 0.9601773	test: 0.8877507	best: 0.8878121 (4824)	total: 5m 22s	remaining: 5m 22s
5600:	learn: 0.9611410	test: 0.8877454	best: 0.8878121 (4824)	total: 5m 28s	remaining: 5m 16s
5700:	learn: 0.9620526	test: 0.8877361	best: 0.8878121 (4824)	t

In [ ]:
submission = pd.DataFrame({'encounter_id':test.encounter_id,TARGET_COL:test_preds})
submission.to_csv('Submissions/submission_{0}.csv'.format(model_name),index=False)

In [ ]:
explainer = shap.TreeExplainer(model_cv_fitted)
shap_values_test = explainer.shap_values(Pool(test_set))
shap.summary_plot(shap_values_test, test_copy[model_cv_fitted.feature_names_])

In [ ]:
shap.dependence_plot('num_na_demographic', shap_values_test, test_copy)

In [ ]:
shap_values_valid = explainer.shap_values(valid_set)
shap.summary_plot(shap_values_valid, validX_cv[model_cv_fitted.feature_names_])